In [1]:
import numpy as np
import string
from nltk.corpus import stopwords
import csv
import pandas as pd

In [2]:
url = "./datasets/pre_processed_dataset.csv"
names = ["tweet"]
data = pd.read_csv(url,names=names)
data = data.drop(data.index[0])

In [3]:
from nltk.tokenize import word_tokenize
import nltk

sentences = data.tweet

stop_words = nltk.corpus.stopwords.words('english')
data_set_tokenized = []

for i in sentences:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    data_set_tokenized.append(filtered_sentence)
# filtered_sentence

## Word2Vec with skipgramc

In [4]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec


In [5]:

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.get_latest_training_loss = 0
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss() - self.get_latest_training_loss
        self.get_latest_training_loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1


In [12]:
skip_gram_model = Word2Vec(sentences=data_set_tokenized, vector_size=50, window=2, min_count=4, workers=2, epochs=200, sg=1,
        compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 3456.19384765625
Loss after epoch 1: 3466.2734375
Loss after epoch 2: 3462.94287109375
Loss after epoch 3: 3310.583984375
Loss after epoch 4: 3521.51953125
Loss after epoch 5: 3466.419921875
Loss after epoch 6: 3604.15625
Loss after epoch 7: 3341.4375
Loss after epoch 8: 3385.228515625
Loss after epoch 9: 3567.513671875
Loss after epoch 10: 3133.34375
Loss after epoch 11: 3023.7734375
Loss after epoch 12: 2946.80859375
Loss after epoch 13: 2721.5078125
Loss after epoch 14: 2659.58203125
Loss after epoch 15: 2718.29296875
Loss after epoch 16: 2428.05078125
Loss after epoch 17: 2391.83203125
Loss after epoch 18: 2318.3046875
Loss after epoch 19: 2405.046875
Loss after epoch 20: 2153.23046875
Loss after epoch 21: 2216.98828125
Loss after epoch 22: 2357.921875
Loss after epoch 23: 2370.359375
Loss after epoch 24: 2359.6015625
Loss after epoch 25: 2192.328125
Loss after epoch 26: 2423.921875
Loss after epoch 27: 2312.1484375
Loss after epoch 28: 2195.2109375
Loss after e

In [7]:
skip_gram_model.wv.most_similar('gmo',topn=10)

[('food', 0.47279417514801025),
 ('kenyans', 0.4098816514015198),
 ('us', 0.40324679017066956),
 ('lifting', 0.40212175250053406),
 ('longterm', 0.39845240116119385),
 ('foods', 0.39564260840415955),
 ('called', 0.3852192759513855),
 ('read', 0.3750094175338745),
 ('going', 0.3703593909740448),
 ('old', 0.3680415153503418)]

In [8]:
skip_gram_model.wv.most_similar('gmo',topn=10)

[('food', 0.47279417514801025),
 ('kenyans', 0.4098816514015198),
 ('us', 0.40324679017066956),
 ('lifting', 0.40212175250053406),
 ('longterm', 0.39845240116119385),
 ('foods', 0.39564260840415955),
 ('called', 0.3852192759513855),
 ('read', 0.3750094175338745),
 ('going', 0.3703593909740448),
 ('old', 0.3680415153503418)]

In [9]:
skip_gram_model.wv.most_similar('corn',topn=10)

[('mexico', 0.7479697465896606),
 ('american', 0.6770910620689392),
 ('report', 0.6595668792724609),
 ('effects', 0.6565154194831848),
 ('products', 0.6527459025382996),
 ('fed', 0.6449334025382996),
 ('ban', 0.6268152594566345),
 ('long', 0.6093230247497559),
 ('modified', 0.6037793159484863),
 ('rich', 0.5912718772888184)]

## Word2Vec with CBOW

In [10]:
cbow_model = Word2Vec(sentences=data_set_tokenized, vector_size=100, window=10, min_count=4, workers=2, epochs=5, sg=0,
        compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 2229.346923828125
Loss after epoch 1: 2304.605712890625
Loss after epoch 2: 2364.0224609375
Loss after epoch 3: 2250.18994140625
Loss after epoch 4: 2307.58984375


In [11]:
cbow_model.wv.most_similar('crop',topn=10)

[('war', 0.3049982786178589),
 ('chemicals', 0.21580922603607178),
 ('labeling', 0.20013035833835602),
 ('news', 0.1575966775417328),
 ('say', 0.15510551631450653),
 ('also', 0.15459318459033966),
 ('herbicide', 0.1451404094696045),
 ('seeds', 0.1409464329481125),
 ('scientists', 0.1345074325799942),
 ('chestnut', 0.13428443670272827)]

## LSTM